In [1]:
'''
model cell
'''
# import the necessary libraries
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution1D
from tensorflow.keras.layers import Dropout, Input, BatchNormalization, MaxPooling1D, concatenate
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn import preprocessing
import joblib


import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import time
import datetime
import plotCEG


from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"]="2,3" 

# Declare DL Model

In [ ]:

def Deduction_Learning(include_top=True, h_activation = 'relu', output_activation = 'linear', learn_rate = 0.001, 
                       beta2 = 0.95, loss_param = 'LogCosh'):
    '''
    include_top: whether to include the fully-connected layers 
    input_shape : the deep learning inputs from the input PPG signal
    output_activation : the last regressin layer activation
    loss_param :our argument to compile the training model
    loss_param = 'mean_squared_error'
    '''
    input_shape=Input(shape=(403, 2),name= "net_input")
    # Block 1
    x = Convolution1D(filters=256, kernel_size=5, name="block1_conv1")(input_shape)
    x = BatchNormalization(axis=-1, name="block1_batchnorm")(x)
    x = Activation(h_activation, name="block1_act")(x)
    x = MaxPooling1D(pool_size=2, name="block1_pool")(x)
    # Block 2
    x = Convolution1D(filters=256, kernel_size=5, name="block2_conv1") (x)
    x = BatchNormalization(axis=-1, name="block2_batchnorm")(x)
    x = Activation(h_activation, name="block2_act")(x)
    x = MaxPooling1D(pool_size=2, name="block2_pool")(x)
    # Block 3
    x = Convolution1D(filters=512, kernel_size=5, name="block3_conv1") (x)
    x = BatchNormalization(axis=-1, name="block3_batchnorm")(x)
    x = Activation(h_activation, name="block3_act")(x)
    x = MaxPooling1D(pool_size=2, name="block3_pool")(x)
    # Block 4
    x = Convolution1D(filters=1024, kernel_size=5, name="block4_conv1") (x)
    x = BatchNormalization(axis=-1, name="block4_batchnorm")(x)
    x = Activation(h_activation, name="block4_act")(x)
    x = MaxPooling1D(pool_size=2, name="block4_pool")(x)

    if include_top:
        x = Flatten(name="flatten")(x)
        input1 = keras.layers.Input(shape=(2,))
        x = BatchNormalization(axis=-1, name = "batch_norm")(x)
        x = keras.layers.Concatenate(axis=-1)([x, input1])
        x = Dense(1024,activation=h_activation, name="FC1")(x)
        x = Dense(512,activation=h_activation, name="FC2")(x)
        
        x = Dense(1,activation=output_activation, name="predictions")(x)
    # Create model.
    model = Model(inputs=[input_shape,input1], outputs=[x])
    
    Optimizer=Adam(lr=learn_rate, beta_1=0.9, beta_2=beta2)
    model.compile(loss=loss_param, optimizer=Optimizer)
    return model


def get_date_delta(base_pid,pred_pid):
    base_pid_path = path_indexer_df.loc[path_indexer_df['Person No']==base_pid]['ECGidx'].values[0]
    yymmdd = base_pid_path.split("/")[-2]
    first_day = datetime.datetime.strptime(yymmdd,'%Y%m%d')
    
    pred_path = path_indexer_df.loc[path_indexer_df['Person No']==pred_pid]['ECGidx'].values[0]
    yymmdd = pred_path.split("/")[-2]
    datetime_object  = datetime.datetime.strptime(yymmdd,'%Y%m%d')
    delta = datetime_object - first_day
    return delta.days

# File path input

In [3]:
merge_all = pd.read_csv("/data/dp2/data/justin/data/merge_all_norm_resp_20191203-20.csv")
merge_all = merge_all.loc[merge_all['Time']==0]

path_indexer_df = pd.read_csv("file_path_index_2023_Norm.csv")

final_df = pd.read_pickle('/data/dp2/data/justin/Sensor_Journal_invitation/Sensor_processedData_All.pkl')  #for imaginary HbA1c
final_df=final_df.set_index('person_time_idx')
final_df.head()

implicit_df = pd.read_pickle("2024_ImplicitHbA1c_set1.pkl")



In [4]:
BL_drug_cond=(merge_all['BL_drug']==0)
BP_drug_cond=(merge_all['BP_drug']==0)
DM_drug_cond=(merge_all['DM_drug']==0)
DM_inject_cond=(merge_all['DM_inject']==0)
O_drug_cond=(merge_all['O_drug']==0)

noDM_drug_id = merge_all.loc[(merge_all['Time']==0)&(DM_drug_cond)&DM_inject_cond]['person_time_idx']  #不含糖尿藥物

DM_drug_id = merge_all.loc[(merge_all['Time']==0)&
                           BL_drug_cond&BP_drug_cond&
                           (~DM_drug_cond)&DM_inject_cond&
                           O_drug_cond]['person_time_idx']  #不含胰島素

anyDM_drug_id=merge_all.loc[(merge_all['Time']==0)&(~BL_drug_cond|~BP_drug_cond|~O_drug_cond)&
                           (~DM_drug_cond|~DM_inject_cond)]['person_time_idx'] # 有服用血糖藥 或 有胰島素注射 同時混用其他藥物


set1_target_Base_A = merge_all.loc[(merge_all['count_time']>=2)&(merge_all['count_time']<3)&
                     (merge_all['HbA1C']>6.5)&
                     (merge_all['person_time_idx'].isin(noDM_drug_id))]['count'].unique()

set1_target_Base_B = merge_all.loc[(merge_all['count_time']>=3)&
                     (merge_all['person_time_idx'].isin(noDM_drug_id))]['count'].unique()
set1_target_Base =np.concatenate([set1_target_Base_A,set1_target_Base_B])

print(set1_target_Base )

set2_target_Base_pids = list( merge_all.loc[(merge_all['count_time']>=3)&(merge_all['person_time_idx'].isin(DM_drug_id))]['count'].unique())


set3_target_Base_pids = list( merge_all.loc[(merge_all['count_time']>=3)&(merge_all['person_time_idx'].isin(anyDM_drug_id))]['count'].unique())

print("SET 1 符合條件受測者共 {} 人".format(len(set1_target_Base)))
print(set1_target_Base )

In [54]:
merge_all.loc[(merge_all['Time']==0)&
                           BL_drug_cond&BP_drug_cond&
                           (~DM_drug_cond)&DM_inject_cond&
                           O_drug_cond&(merge_all['count']==0)]['Person No'].unique()

In [24]:
set1_df=pd.read_pickle("/data/dp2/data/2023_NIBG_with_FFT_input/set_1_paired_PPG_waveform+feature.pkl") 


set1_df['pair'] = set1_df['person_time_idx_base']+'_'+set1_df['person_time_idx_test']

set1_target = merge_all.loc[merge_all['count'].isin(set1_target_Base)|merge_all['Person No'].isin(set1_target_Base)]['person_time_idx']

set1_df = set1_df.loc[(set1_df['person_time_idx_base'].isin(set1_target))|set1_df['person_time_idx_test'].isin(set1_target)]


In [13]:
img_HbA1c_lst=[]

for person_time_idx in set1_df['person_time_idx_base']:
    img_HbA1c_lst.append(implicit_df.loc[implicit_df['person_time_idx_base']==person_time_idx]['implicit_HbA1c'].values[0])
    
set1_df['img_HbA1c']=img_HbA1c_lst

set1_df.head()

In [ ]:

col = list(np.linspace(0,399,400).astype(int).astype(str))
col=col+['HR','FW_50','Total_Area']
base_col = [c+"_base" for c in col]
test_col = [c+"_test" for c in col]

for target_pid in set1_target_Base:
    test_pid_t_idx = merge_all.loc[(merge_all['Person No']==target_pid)|(merge_all['count']==target_pid)]['person_time_idx'] #get test set pid_t_idx
    
    
    X_train_s      = set1_df.loc[((set1_df['person_time_idx_base'].isin(noDM_drug_id))|(set1_df['person_time_idx_test'].isin(noDM_drug_id)))&
                                (~set1_df['person_time_idx_test'].isin(test_pid_t_idx))&
                                (~set1_df['person_time_idx_base'].isin(test_pid_t_idx))][test_col]
    X_train_s_base = set1_df.loc[((set1_df['person_time_idx_base'].isin(noDM_drug_id))|(set1_df['person_time_idx_test'].isin(noDM_drug_id)))&
                                (~set1_df['person_time_idx_test'].isin(test_pid_t_idx))&
                                (~set1_df['person_time_idx_base'].isin(test_pid_t_idx))][base_col]


    X_test_s      = set1_df.loc[((set1_df['person_time_idx_base'].isin(test_pid_t_idx))|(set1_df['person_time_idx_test'].isin(test_pid_t_idx)))][test_col]
    X_test_s_base = set1_df.loc[((set1_df['person_time_idx_base'].isin(test_pid_t_idx))|(set1_df['person_time_idx_test'].isin(test_pid_t_idx)))][base_col]
    
    t = np.array([50,400])
    t = t.reshape(-1, 1)
    scalert = preprocessing.MinMaxScaler().fit(t)

    Y_train = set1_df.loc[((set1_df['person_time_idx_base'].isin(noDM_drug_id))|(set1_df['person_time_idx_test'].isin(noDM_drug_id)))&
                        (~set1_df['person_time_idx_test'].isin(test_pid_t_idx))&
                        (~set1_df['person_time_idx_base'].isin(test_pid_t_idx))]['BG_test']
    Y_train = np.array(Y_train).reshape(-1,1)
    Y_train = scalert.transform(Y_train)

    Y_test = set1_df.loc[((set1_df['person_time_idx_base'].isin(test_pid_t_idx))|(set1_df['person_time_idx_test'].isin(test_pid_t_idx)))]['BG_test']
    Y_test = np.array(Y_test).reshape(-1,1)
    Y_test = scalert.transform(Y_test)
    
    # reshape train data
    X_train_r_s = np.zeros((len(X_train_s), 403, 2))
    X_train_r_s[:, :, 0] = X_train_s
    X_train_r_s[:, :, 1] = X_train_s_base

    # reshape vali data
    X_test_r_s = np.zeros((len(X_test_s), 403, 2))
    X_test_r_s[:, :, 0] = X_test_s
    X_test_r_s[:, :, 1] = X_test_s_base
    
    features=['BG_base','img_HbA1c']
 
    # reshape train data
    X_train_r_f = set1_df.loc[((set1_df['person_time_idx_base'].isin(noDM_drug_id))|(set1_df['person_time_idx_test'].isin(noDM_drug_id)))&
                            (~set1_df['person_time_idx_test'].isin(test_pid_t_idx))&
                            (~set1_df['person_time_idx_base'].isin(test_pid_t_idx))][features]

    # reshape vali data
    X_test_r_f = set1_df.loc[((set1_df['person_time_idx_base'].isin(test_pid_t_idx))|(set1_df['person_time_idx_test'].isin(test_pid_t_idx)))][features]

    
    ############################
    start = time.time() # Mark start trining time

    keras.backend.clear_session()

    #fit
    model= Deduction_Learning()
    history = model.fit([X_train_r_s,X_train_r_f], Y_train, epochs=75, batch_size=2000,
                        validation_data=([X_test_r_s,X_test_r_f], Y_test),verbose=0)

    end = time.time() # Mark end trining time
    # 輸出結果
    print("執行時間：%f 分鐘" % ((int(end) - int(start))/60))
    
    plt.figure(figsize=(6,2)) #plot training & testing loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylim(0,0.1)
    plt.show()

    #build testing result table
    test_pred = model.predict([X_test_r_s,X_test_r_f])
    test_pred1 = scalert.inverse_transform(test_pred)
    test_pred1=test_pred1.reshape(-1)

    train_pred  =model.predict([X_train_r_s,X_train_r_f])
    train_pred1 = scalert.inverse_transform(train_pred)
    train_pred1=train_pred1.reshape(-1)

    Y_test = set1_df.loc[((set1_df['person_time_idx_base'].isin(test_pid_t_idx))|(set1_df['person_time_idx_test'].isin(test_pid_t_idx)))][['pair','BG_test']]
    Y_test['pred_BG']=test_pred1

    ref_BG=[]
    pred_BG=[]
    std=[]
    ptile_25=[]
    ptile_75=[]
    delta_days=[]
    for pair in Y_test['pair'].unique():
        ref_BG.append(int(set1_df.loc[set1_df['pair']==pair]['BG_test'].values[0])) 
        pred_BG.append(Y_test.loc[Y_test['pair']==pair]['pred_BG'].median()) 
        std.append(Y_test.loc[Y_test['pair']==pair]['pred_BG'].std()) 
        ptile_25.append(Y_test.loc[Y_test['pair']==pair]['pred_BG'].describe()['25%']) 
        ptile_75.append(Y_test.loc[Y_test['pair']==pair]['pred_BG'].describe()['75%']) 

        base_pid = int(pair.split('_')[0])
        test_pid = int(pair.split('_')[2])
        delta_days.append(get_date_delta(base_pid,test_pid))

    test_result_df = pd.DataFrame({'pair': list(Y_test['pair'].unique()), 'ref_BG':ref_BG, 'pred_BG':pred_BG , 'std':std, "ptile_25":ptile_25, "ptile_75":ptile_75,'delta_days':delta_days})  

    plotCEG.plot_test(list(test_result_df['ref_BG']),list(test_result_df['pred_BG']),'Testing set {}'.format(target_pid))
    
    #model.save("temp_model_holder.h5")

In [ ]:
test_result_df.to_pickle("./result_tables/2024_ImplicitDL_set1-{}".format(target_pid)) # Save test result table
